<a href="https://colab.research.google.com/github/qingtaogh/BigData/blob/main/colab/MLearingNotbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Case study

In [24]:

# install pyspark using pip
!pip install --ignore-install -q pyspark
# install findspark using pip
!pip install --ignore-install -q findspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
result = 8 ** 4
print(result)

4096


In [3]:

print("Split this string".split())

['Split', 'this', 'string']


In [4]:
planet = "Earth"
diameter = 12742

output = "The diameter of {} is {} kilometers.".format(planet, diameter)
print(output)

The diameter of Earth is 12742 kilometers.


In [6]:
the_list = [1, 2, [3, 4], [5, [100, 200, ['target']], 23, 11], 1, 7]

result = the_list[3][1][2][0]
print(result)

target


In [7]:
the_dic = {
    'k1': [1, 2, 3,
          {'tricky': ['oh', 'man', 'inception',
                      {'target': [1, 2, 3, 'hello']}
                    ]
          }
    ]
}


word = the_dic['k1'][3]['tricky'][3]['target'][3]
print(word)

hello


In [20]:
def contains_elephant(input_string):
    return 'elephant' in input_string.lower()

test_string = "I saw an Elephant. today."
result = contains_elephant(test_string)
print(result)

True


In [21]:
import re
def count_elephants(input_string):

    words = re.split(r'[ .]+', input_string)
    count = words.count('elephant')
    return count

test_string = "I saw an elephant and another elephant."

print(count_elephants(test_string))


2


In [22]:
def speed_category(speed, is_birthday):

    if is_birthday:
        speed += 5


    if speed <= 60:
        return "Low speed"
    elif 61 <= speed <= 80:
        return "Medium speed"
    else:
        return "Fast speed"


print(speed_category(60, False))
print(speed_category(65, False))
print(speed_category(81, False))
print(speed_category(60, True))
print(speed_category(80, True))

Low speed
Medium speed
Fast speed
Medium speed
Fast speed


In [31]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Employee Data") \
    .getOrCreate()

# Define the data
data = [
    ("James", "Sales", "SG", 70000, 34, 10000),
    ("Michael", "Sales", "SG", 66000, 56, 20000),
    ("Robert", "Sales", "MY", 61000, 30, 23000),
    ("Maria", "Finance", "MY", 60000, 24, 23000),
    ("Raman", "Finance", "USA", 79000, 40, 24000),
    ("Scott", "Finance", "USA", 63000, 36, 19000),
    ("Jen", "Finance", "UK", 89000, 53, 15000),
    ("Jeff", "Marketing", "UK", 70000, 25, 18000),
    ("Alice", "Marketing", "UK", 78000, 50, 21000),
    ("Ada", "IT", "SG", 83000, 35, 11000),
    ("Jackson", "IT", "MY", 71000, 30, 21000),
    ("Cooper", "IT", "UK", 91000, 40, 21000)
]

# Define the schema (column names)
columns = ["employee_name", "department", "country", "salary", "age", "bonus"]

# Create the DataFrame
df = spark.createDataFrame(data, schema=columns)

# Show the DataFrame
df.show()

df.printSchema()

df.describe("salary", "age", "bonus").show()

+-------------+----------+-------+------+---+-----+
|employee_name|department|country|salary|age|bonus|
+-------------+----------+-------+------+---+-----+
|        James|     Sales|     SG| 70000| 34|10000|
|      Michael|     Sales|     SG| 66000| 56|20000|
|       Robert|     Sales|     MY| 61000| 30|23000|
|        Maria|   Finance|     MY| 60000| 24|23000|
|        Raman|   Finance|    USA| 79000| 40|24000|
|        Scott|   Finance|    USA| 63000| 36|19000|
|          Jen|   Finance|     UK| 89000| 53|15000|
|         Jeff| Marketing|     UK| 70000| 25|18000|
|        Alice| Marketing|     UK| 78000| 50|21000|
|          Ada|        IT|     SG| 83000| 35|11000|
|      Jackson|        IT|     MY| 71000| 30|21000|
|       Cooper|        IT|     UK| 91000| 40|21000|
+-------------+----------+-------+------+---+-----+

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- country: string (nullable = true)
 |-- salary: long (nullable = true)


In [27]:
from pyspark.sql import functions as F
# Perform groupBy and aggregation
aggregated_df = df.groupBy("department").agg(
    F.min("salary").alias("min_salary"),
    F.max("salary").alias("max_salary"),
    F.avg("salary").alias("avg_salary"),
    F.sum("salary").alias("total_salary")
)

# Show the aggregated results
aggregated_df.show()

+----------+----------+----------+-----------------+------------+
|department|min_salary|max_salary|       avg_salary|total_salary|
+----------+----------+----------+-----------------+------------+
|     Sales|     61000|     70000|65666.66666666667|      197000|
|   Finance|     60000|     89000|          72750.0|      291000|
| Marketing|     70000|     78000|          74000.0|      148000|
|        IT|     71000|     91000|81666.66666666667|      245000|
+----------+----------+----------+-----------------+------------+



In [28]:
# Perform groupBy and aggregation by country
aggregated_by_country_df = df.groupBy("country").agg(
    F.min("salary").alias("min_salary"),
    F.max("salary").alias("max_salary"),
    F.avg("salary").alias("avg_salary"),
    F.sum("salary").alias("total_salary")
)

# Show the aggregated results
aggregated_by_country_df.show()

+-------+----------+----------+----------+------------+
|country|min_salary|max_salary|avg_salary|total_salary|
+-------+----------+----------+----------+------------+
|     MY|     60000|     71000|   64000.0|      192000|
|    USA|     63000|     79000|   71000.0|      142000|
|     SG|     66000|     83000|   73000.0|      219000|
|     UK|     70000|     91000|   82000.0|      328000|
+-------+----------+----------+----------+------------+



In [34]:

aggregated_by_age_df = df.groupBy("age").agg(
    F.min("salary").alias("min_salary"),
    F.max("salary").alias("max_salary"),
    F.avg("salary").alias("avg_salary"),
    F.sum("salary").alias("total_salary")
)


aggregated_by_age_df.orderBy(F.desc("age")).show()

+---+----------+----------+----------+------------+
|age|min_salary|max_salary|avg_salary|total_salary|
+---+----------+----------+----------+------------+
| 56|     66000|     66000|   66000.0|       66000|
| 53|     89000|     89000|   89000.0|       89000|
| 50|     78000|     78000|   78000.0|       78000|
| 40|     79000|     91000|   85000.0|      170000|
| 36|     63000|     63000|   63000.0|       63000|
| 35|     83000|     83000|   83000.0|       83000|
| 34|     70000|     70000|   70000.0|       70000|
| 30|     61000|     71000|   66000.0|      132000|
| 25|     70000|     70000|   70000.0|       70000|
| 24|     60000|     60000|   60000.0|       60000|
+---+----------+----------+----------+------------+

